In [ ]:
# Embedding : Glove
# Input : Padded Sequence of Interger Vlaues from Keras tokenizer
# Output : One hot Encoder of the classes

In [1]:
import pandas as pd

data = pd.read_csv("C:\MyWork\MyLearning\Career Growth\ML\Files\DataSet\Consumer Complaint.csv")

data = data[['Product','Issue']]

In [2]:
data.head()

,Product,Issue
0,Debt collection,Communication tactics
1,Debt collection,Cont'd attempts collect debt not owed
2,Mortgage,"Application, originator, mortgage broker"
3,Credit card,Other
4,Debt collection,Cont'd attempts collect debt not owed


In [3]:
# Choose Not null values
data = data[pd.notnull(data['Issue'])]

In [4]:
# No null values
data.isnull().sum()

Product    0
Issue      0
dtype: int64

In [5]:
data.shape

(28154, 2)

In [6]:
from bs4 import BeautifulSoup
import re
from string import punctuation

punctns = punctuation + "'"

def cleanText(text):
    text = ''.join([x for x in text if x not in punctuation])
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    text = text.lower()
    text = text.replace('x', '')
    return text

In [7]:
data['Issue'] = data['Issue'].apply(cleanText)

In [8]:
# Find Vocabulary Size
data['Issue'].apply(lambda x: len(x.split(' '))).sum()

122193

In [9]:
data.head()

,Product,Issue
0,Debt collection,communication tactics
1,Debt collection,contd attempts collect debt not owed
2,Mortgage,application originator mortgage broker
3,Credit card,other
4,Debt collection,contd attempts collect debt not owed


In [10]:
data.Product.value_counts()

Debt collection            7494
Mortgage                   6612
Credit reporting           5717
Credit card                2830
Bank account or service    2607
Consumer loan              1314
Student loan                785
Payday loan                 348
Money transfers             232
Prepaid card                175
Other financial service      40
Name: Product, dtype: int64

In [53]:
doc = [x.split() for x in data['Issue']]

max_seq_length = max([len(x) for x in data['Issue']])

max_seq_length

40

In [37]:
# Find the Vocabulari size
len(set([x for y in doc for x in y]))

186

In [84]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import sequence
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = 8000,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['Issue'].values)
sequence = tokenizer.texts_to_sequences(data['Issue'].values)
word2idx_ = tokenizer.word_index

In [85]:
list(word2idx_.items())[:10]

[('loan', 1),
 ('credit', 2),
 ('debt', 3),
 ('or', 4),
 ('account', 5),
 ('report', 6),
 ('incorrect', 7),
 ('information', 8),
 ('on', 9),
 ('not', 10)]

In [86]:
sequence[:10]

[[23, 24],
 [11, 12, 13, 3, 10, 14],
 [56, 58, 59, 60],
 [52],
 [11, 12, 13, 3, 10, 14],
 [23, 24],
 [37, 32, 1, 4, 31],
 [23, 24],
 [11, 12, 13, 3, 10, 14],
 [63, 64, 65, 66]]

In [63]:
X = pad_sequences(sequence,maxlen=130)

In [64]:
X.shape

(28154, 130)

In [65]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y = ohe.fit_transform(data[['Product']]).toarray()

y.shape

(28154, 11)

In [66]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open(r'C:\MyWork\MyLearning\Career Growth\ML\Files\DataSet\glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')    
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [67]:
num_words = len(word2idx_) + 1

num_words

187

In [68]:
import numpy as np

EMBEDDING_SIZE = 100
embedding_matrix = np.zeros((num_words, EMBEDDING_SIZE))

embedding_matrix.shape

(187, 100)

In [89]:
X[:1]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        23, 24]])

In [69]:
for word, index in word2idx_.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [72]:
embedding_matrix.shape

(187, 100)

In [75]:
from keras.models import Sequential
from keras.layers import Embedding

model = Sequential()
# weights = [embedding_matrix] : Using Pretrained Model
model.add(Embedding(num_words,100,weights = [embedding_matrix],input_length=130,trainable = False))
model.add(LSTM(64,dropout=0.7,recurrent_dropout=0.7))
model.add(Dense(11,activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [76]:
history = model.fit(X, y, epochs=30, batch_size=256,validation_split=0.2,
                    callbacks=[EarlyStopping(monitor='val_loss',patience=7, min_delta=0.0001)])


Train on 22523 samples, validate on 5631 samples
Epoch 1/30
22523/22523 [==============================] - 21s 913us/step - loss: 1.3847 - acc: 0.6029 - val_loss: 0.5644 - val_acc: 0.8521
Epoch 2/30
22523/22523 [==============================] - 38s 2ms/step - loss: 0.5519 - acc: 0.8632 - val_loss: 0.2155 - val_acc: 0.9666
Epoch 3/30
22523/22523 [==============================] - 42s 2ms/step - loss: 0.3337 - acc: 0.9213 - val_loss: 0.1315 - val_acc: 0.9691
Epoch 4/30
22523/22523 [==============================] - 41s 2ms/step - loss: 0.2396 - acc: 0.9432 - val_loss: 0.0961 - val_acc: 0.9741
Epoch 5/30
22523/22523 [==============================] - 39s 2ms/step - loss: 0.1863 - acc: 0.9555 - val_loss: 0.0745 - val_acc: 0.9780
Epoch 6/30
22523/22523 [==============================] - 41s 2ms/step - loss: 0.1469 - acc: 0.9644 - val_loss: 0.0610 - val_acc: 0.9794
Epoch 7/30
22523/22523 [==============================] - 41s 2ms/step - loss: 0.1246 - acc: 0.9691 - val_loss: 0.0504 - val_a

In [90]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 130, 100)          18700     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_5 (Dense)              (None, 11)                715       
Total params: 61,655
Trainable params: 42,955
Non-trainable params: 18,700
_________________________________________________________________


In [77]:
product_wordIndex = {
                    0:"Bank account or service",
                    1:"Consumer loan",
                    2:"Credit card",
                    3:"Credit reporting",
                    4:"Debt collection",
                    5:"Money transfers",
                    6:"Mortgage",
                    7:"Other financial service",
                    8:"Payday loan",
                    9:"Prepaid card",
                    10:"Student loan"
                    }

def generate_Padsequence(text,maxlen):
    
    # Tokenize the doc
    token = [x for x in text.split()]
    
    # Text to Sequence Converter
    token = [word_index[t] for t in token]
    
    # Sequrnce to padding
    pad_seq = pad_sequences([token],maxlen)
    
    return pad_seq


def predictProduct(model,issue):
    
    issue = cleanText(issue)
    
    # Generate Padded Sequennce
    text_padSequence = generate_Padsequence(issue,130)
    
    # Predict from the model
    pred = model.predict(text_padSequence)
    
    # Flattern the array
    pred = pred.flatten().tolist()
    
    # Get the Max Value
    max_value = max(pred)
    
    # find max value index
    product_index = pred.index(max_value)
    
    return product_wordIndex[product_index]

In [78]:
# choose a random element from a list
from random import seed
from random import choice
# seed random number generator
seed(1)
# prepare a sequence
sequence = [i for i in range(20)]
# make choices from the sequence

index = []

for _ in range(10):
    selection = choice(sequence)
    index.append(selection)
    
actual_output = []
predicted_output = []

df = data.iloc[index,:]

In [79]:
df

,Product,Issue
4,Debt collection,contd attempts collect debt not owed
18,Debt collection,communication tactics
2,Mortgage,application originator mortgage broker
8,Debt collection,contd attempts collect debt not owed
3,Credit card,other
15,Debt collection,disclosure verification of debt
14,Debt collection,disclosure verification of debt
15,Debt collection,disclosure verification of debt
12,Debt collection,contd attempts collect debt not owed
6,Consumer loan,managing the loan or lease


In [80]:

for index, row in df.iterrows():
    
    actual_output.append(row['Product'])
    
    issue = row['Issue']
    
    predicted_output.append(predictProduct(model,issue))

In [81]:
pd.DataFrame({"Actual":actual_output,"Predicted":predicted_output})

,Actual,Predicted
0,Debt collection,Debt collection
1,Debt collection,Debt collection
2,Mortgage,Mortgage
3,Debt collection,Debt collection
4,Credit card,Credit card
5,Debt collection,Debt collection
6,Debt collection,Debt collection
7,Debt collection,Debt collection
8,Debt collection,Debt collection
9,Consumer loan,Consumer loan
